In [1]:
import os
import json
import pandas as pd
import numpy as np

In [7]:
results = dict()

data_dir = "results"
for file_path in os.listdir(data_dir):
    with open(os.path.join(data_dir, file_path)) as f:
        results[file_path[:-5]] = json.load(f)

In [8]:
def get_result_table(statistic_name: str):
    tmp_dict = dict()
    problem_names=list()
    for name, data in results.items():
        problem_name, solver_name = name.split("-")
        if problem_name not in problem_names:
            problem_names.append(problem_name)
        if solver_name not in tmp_dict:
            tmp_dict[solver_name] = []
        tmp_dict[solver_name].append(data[statistic_name])
    return pd.DataFrame(tmp_dict, index=problem_names)

def get_result_table_average(statistic_name: str):
    tmp_dict = dict()
    problem_names=list()
    for name, data in results.items():
        problem_name, solver_name = name.split("-")
        if problem_name not in problem_names:
            problem_names.append(problem_name)
        if solver_name not in tmp_dict:
            tmp_dict[solver_name] = []
        tmp_dict[solver_name].append(
            np.average([solution[statistic_name] for solution in data["solutions"]])
            )
    return pd.DataFrame(tmp_dict, index=problem_names)

In [9]:
avg_times = get_result_table("avg_time")
avg_times.head()

,GreedySearchSolver,HeuristicSolver,RandomSearchSolver,RandomWalkSolver,SteepestSearchSolver
bur26a,0.002201,0.000017,0.002208,0.002206,0.002844
chr12a,0.000070,0.000006,0.000073,0.000072,0.000090
chr22a,0.000731,0.000013,0.000737,0.000735,0.001090
els19,0.000962,0.000011,0.000967,0.000966,0.000778
esc32e,0.000346,0.000022,0.000354,0.000353,0.001047


In [10]:
avg_cost = get_result_table_average("cost")
avg_cost.head()

,GreedySearchSolver,HeuristicSolver,RandomSearchSolver,RandomWalkSolver,SteepestSearchSolver
bur26a,5.445425e+06,5940358.0,5.628761e+06,5.614830e+06,5.444306e+06
chr12a,1.428212e+04,41064.0,2.471650e+04,2.284012e+04,1.391265e+04
chr22a,6.979823e+03,12420.0,1.000913e+04,9.597292e+03,6.996749e+03
els19,2.259623e+07,38627698.0,3.059925e+07,2.914552e+07,2.165328e+07
esc32e,2.000000e+00,44.0,1.454359e+01,1.396825e+01,2.000000e+00


In [11]:
optimum = get_result_table("optimal_cost")
avg_dist_to_optimum = avg_cost / optimum
avg_dist_to_optimum.head()

,GreedySearchSolver,HeuristicSolver,RandomSearchSolver,RandomWalkSolver,SteepestSearchSolver
bur26a,1.003456,1.094660,1.037240,1.034673,1.003250
chr12a,1.495196,4.298995,2.587573,2.391135,1.456517
chr22a,1.133824,2.017544,1.625915,1.559014,1.136574
els19,1.312777,2.244159,1.777729,1.693272,1.257994
esc32e,1.000000,22.000000,7.271793,6.984127,1.000000


In [12]:
avg_evaluated_solutions = get_result_table_average("iteration_counts")
avg_evaluated_solutions.drop("HeuristicSolver", axis=1, inplace=True)
avg_evaluated_solutions.head()

,GreedySearchSolver,RandomSearchSolver,RandomWalkSolver,SteepestSearchSolver
bur26a,5741.868132,722.598234,4618.151982,7206.321023
chr12a,371.476977,76.999053,246.954912,508.200595
chr22a,2241.088321,309.653392,1704.432770,3392.781046
els19,3408.140385,523.722705,2531.384170,2786.661742
esc32e,725.141376,79.871722,604.909700,2194.125523


In [13]:
avg_steps = get_result_table_average("step_count")
avg_steps = avg_steps[["GreedySearchSolver", "SteepestSearchSolver"]]
avg_steps.head()

,GreedySearchSolver,SteepestSearchSolver
bur26a,96.529670,21.173295
chr12a,14.183975,6.700009
chr22a,34.819708,13.687364
els19,56.086538,15.296267
esc32e,6.186657,3.423640
